In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.shape

(57650, 4)

In [4]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [5]:
df.head(10)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \r\nYou're e..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \r\nCouldn't s...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \r\nI'm sitting all ...


In [8]:
df = df.sample(10000)

In [9]:
df = df.drop('link', axis = 1)

In [10]:
df['text'] = df['text'].str.lower().replace(r'\r','', regex = True).replace(r'\n', '', regex = True).replace(r'^\w\s',' ', regex = True)

In [11]:
import nltk
from nltk.stem.porter import PorterStemmer
stemer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stem = [stemer.stem(w) for w in tokens]
    
    return " ".join(stem)

In [12]:
tokenization("you are beautiful, beauty")

'you are beauti , beauti'

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/aditi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer = 'word', stop_words= 'english')
matrix = tfidvector.fit_transform(df['text'])

similarity = cosine_similarity(matrix)

In [17]:
similarity[1]

array([0.01098945, 1.        , 0.06002723, ..., 0.07748826, 0.0029809 ,
       0.00740556])

In [19]:
def recommendation(song_name):
    idx = df[df['song'] == song_name].index[0]
    distance = sorted(list(enumerate(similarity[idx])), reverse = True, key = lambda x:x[1])
    
    songs = []
    for id in distance[1:21]:
        songs.append(df.iloc[id[0]].song)
    return songs

In [20]:
recommendation('Hello Old Friend')

['Cold Beer And Remote Control',
 'Higher And Higher',
 'Faithfully',
 'Another Life',
 "It's Alright",
 'If You Were Mine',
 'Fly Like A Bird',
 'Gypsy',
 'A Love For You',
 'Viva La Vida',
 "I Can't Understand It",
 "When It's Love",
 'Higher Ground',
 'Mr. Self Destruct',
 'Make Tonight All Mine',
 'Midnight Blue',
 'Tightrope',
 'Just Like That',
 'Smile',
 'Oh Well']